In [1]:
!pip install transformers
!pip install torch
!pip install einops

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


### Load dataset

In [2]:
import pandas as pd
import numpy as np

aml = pd.read_csv('../data processing/cancer/aml/combined_data.csv', sep=",", on_bad_lines="skip")
# Transposer les données
data = aml.transpose()
# Extraire la première ligne et l'affecte au nom de la colonne
data.columns = data.iloc[0]
data = data[1:]
# Réinitialiser les index
data.reset_index(drop=True, inplace=True)
data

Unnamed: 0,X..100130426,X..100133144,X..100134869,X..26823,X..340602,X..388795,X..391343,X..391714,X..441362,X..652919,...,hsa.mir.942,hsa.mir.943,hsa.mir.944,hsa.mir.95,hsa.mir.96,hsa.mir.98,hsa.mir.99a,hsa.mir.99b,Survival,Death
0,-16.60964,4.191342,-16.60964,-16.60964,-16.60964,3.402846,-16.60964,-16.60964,-16.60964,5.228819,...,5.782562,-16.60964,0.975214,-16.60964,-16.60964,6.223134,6.332759,9.33715,245.0,1.0
1,-16.60964,-16.60964,2.212292,-0.372748,-0.372748,1.212264,-16.60964,2.797243,-16.60964,5.05359,...,5.488771,0.131231,0.131231,0.131231,-16.60964,7.119903,1.131225,15.971824,212.0,1.0
2,-16.60964,-16.60964,-0.385884,-0.385884,2.784108,2.421536,-16.60964,-16.60964,1.936105,-16.60964,...,2.727764,-16.60964,-16.60964,-16.60964,-16.60964,6.564263,1.920411,12.471875,31.0,1.0
3,0.188287,1.823672,1.631016,-16.60964,-16.60964,1.145619,-16.60964,0.145625,-16.60964,3.302657,...,7.015997,-16.60964,-16.60964,0.993636,3.99363,6.548218,5.315557,14.136214,215.0,1.0
4,-16.60964,-16.60964,1.491089,2.298427,0.491094,1.491089,-16.60964,0.491094,-16.60964,5.948856,...,5.369449,-16.60964,-16.60964,-16.60964,0.36946,7.09737,7.424732,12.50612,393.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,-16.60964,3.840857,3.578529,1.80876,-16.60964,0.808764,-16.60964,0.808764,-16.60964,5.393715,...,5.78523,-16.60964,1.030349,-16.60964,-16.60964,6.42266,8.622799,13.346907,366.0,1.0
166,-0.78411,0.907437,0.685815,2.023259,-16.60964,4.215912,-16.60964,0.800919,-16.60964,1.800873,...,5.147122,-16.60964,-0.042689,-0.042689,-1.042674,6.441113,5.615509,9.689464,NaN,NaN
167,-16.60964,3.483649,1.995018,1.463942,-0.535938,2.633875,-16.60964,0.463947,-16.60964,2.190776,...,5.446844,-16.60964,1.359387,-16.60964,0.359392,7.498932,3.529307,13.177763,212.0,0.0
168,-16.60964,2.662436,1.005551,0.35942,1.359358,2.818812,-16.60964,0.35942,-16.60964,6.713585,...,4.962938,-16.60964,-16.60964,1.319087,0.319093,7.100441,6.948438,11.100441,334.0,0.0


### DNABERT / HyenaDNA / NT

In [3]:
from transformers.models.bert.configuration_bert import BertConfig
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
import torch

dna_tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True) # LLM: "nucleotide-transformer"
dna_config = BertConfig.from_pretrained("zhihan1996/DNABERT-2-117M")
dna_model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True, config=dna_config)

dna = "ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC"
inputs = dna_tokenizer(dna, return_tensors = 'pt')["input_ids"] # convert DNA sequence to token ID
hidden_states = dna_model(inputs)[0] # [1, sequence_length, 768]

# embedding with mean pooling
embedding_mean = torch.mean(hidden_states[0], dim=0) # use mean embedding of all tokens as the sequence's embedding
print(embedding_mean.shape) # 768

# embedding with max pooling
embedding_max = torch.max(hidden_states[0], dim=0)[0]
print(embedding_max.shape) # 768

C:\Users\OSCAR\.cache\huggingface\modules\transformers_modules\zhihan1996\DNABERT-2-117M\d064dece8a8b41d9fb8729fbe3435278786931f1\bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at zhihan1996/DNABERT-2-117M and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([768])
torch.Size([768])


In [4]:
# Check vocabulary
print("Vocabulary size: ", len(dna_tokenizer.vocab))
print("Top 10 vocabulary: ", list(dna_tokenizer.vocab.items())[:10])

word = "The" 
dna_vocab = dna_tokenizer.get_vocab()
if word in dna_vocab:
    print(f"'{word}' is in the vocabulary {dna_vocab[word]}")
else:
    print(f"'{word}' is not in the vocabulary")

# Check if the input text is a complete word in the vocabulary
tokenized = dna_tokenizer.tokenize(word)
if len(tokenized) == 1 and tokenized[0] == word:
    print(f"'{word}' is a complete word in the vocabulary")
else:
    print(f"'{word}' is splitted: {tokenized}")

Vocabulary size:  4096
Top 10 vocabulary:  [('GTCAGAA', 3606), ('GTGGGTT', 2424), ('TGACTT', 554), ('GCAGTGA', 815), ('GGACAGG', 1710), ('GAAGGAA', 1273), ('GCTTGTG', 3578), ('GTTGTGA', 2012), ('GAGGGCA', 3888), ('GACTA', 308)]
'The' is not in the vocabulary
'The' is splitted: ['T', '[UNK]', '[UNK]']


### BioBERT

In [5]:
# convert dataframe to string
print(data.iloc[0, :-2]) # remove the last two columns (label)
embedding_text = data.iloc[0, :-2].apply(lambda x: f"{x}").to_string()
print()
print(embedding_text)

Unnamed: 0
X..100130426   -16.60964
X..100133144    4.191342
X..100134869   -16.60964
X..26823       -16.60964
X..340602      -16.60964
                  ...   
hsa.mir.95     -16.60964
hsa.mir.96     -16.60964
hsa.mir.98      6.223134
hsa.mir.99a     6.332759
hsa.mir.99b      9.33715
Name: 0, Length: 11705, dtype: object

Unnamed: 0
X..100130426              -16.609640474436812
X..100133144                4.191342344959819
X..100134869              -16.609640474436812
X..26823                  -16.609640474436812
X..340602                 -16.609640474436812
X..388795                    3.40284630657414
X..391343                 -16.609640474436812
X..391714                 -16.609640474436812
X..441362                 -16.609640474436812
X..652919                   5.228819075214507
X..728603                 -16.609640474436812
X..728788                  1.9434375348274688
X..729884                 -16.609640474436812
X..90288                  -16.609640474436812
A2BP1.54715         

In [6]:
bio_tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2") 
bio_model = AutoModelForMaskedLM.from_pretrained("dmis-lab/biobert-base-cased-v1.2")

# test
multiomics_text = embedding_text

# Strategy1: add truncation=True to avoid exceeding the maximum input length 512 (number of token)
inputs = bio_tokenizer(multiomics_text, return_tensors="pt", truncation=True, max_length=512)

# inference
with torch.no_grad():
    outputs = bio_model(**inputs, output_hidden_states=True)
# take the last Transformer Encoder layer's output
last_hidden_state = outputs.hidden_states[-1]  # [batch_size, sequence_length, hidden_size]
# [CLS] token[0] represent the whole sentence's embedding
cls_embedding = last_hidden_state[:, 0, :]  # [batch_size, hidden_size]
print("CLS embedding vector shape:", cls_embedding.shape)
print("CLS embedding vector:", cls_embedding)

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.de

CLS embedding vector shape: torch.Size([1, 768])
CLS embedding vector: tensor([[-3.5796e-01,  1.4439e-02, -6.0701e-01, -4.7969e-01, -3.8862e-01,
         -5.8802e-03,  6.8519e-01, -7.9516e-01, -6.0807e-01, -8.1944e-01,
         -6.8745e-01, -1.2059e-01,  3.4115e-01, -6.5503e-01, -5.2390e-01,
          2.0753e-01,  1.0203e+00, -2.4034e-01, -3.9484e-01, -1.9092e-01,
          3.0531e-02, -9.1315e-01,  2.8889e-01, -2.6653e-02, -2.3281e-01,
         -9.7365e-02,  6.5690e-01,  1.0820e-01, -5.2392e-01,  1.8035e-01,
         -2.7056e-01,  2.3090e-01, -6.3618e-01, -7.9012e-01,  2.1289e-01,
          4.0718e-01,  1.4461e-01,  1.3972e-01,  7.1468e-02,  3.6418e-01,
         -6.5256e-01,  3.5539e-01,  1.3513e-01,  1.8696e-01,  1.4783e-01,
         -6.9705e-01, -1.7809e-01,  1.5231e-01, -2.8421e-01,  5.7187e-02,
          2.1026e-01, -3.3008e-01,  1.2712e-01,  2.9258e-02, -4.7917e-01,
         -6.3653e-01, -1.7980e-01, -3.1182e-01, -1.2180e-01, -9.8340e-02,
          2.1592e-01,  6.7900e-01, -2.234

In [7]:
# # Strategy2: split the input text into chunks and finally concatenate the embedding of each chunk
# chunk_size = 512
# tokens = bio_tokenizer(multiomics_text, return_tensors="pt")["input_ids"].squeeze(0)
# chunks = tokens.split(chunk_size)
# embeddings = []
# with torch.no_grad():
#     for chunk in chunks:
#         chunk_inputs = {"input_ids": chunk.unsqueeze(0)}
#         outputs = bio_model(**chunk_inputs, output_hidden_states=True)
#         cls_embedding = outputs.hidden_states[-1][:, 0, :]
#         embeddings.append(cls_embedding)
# # combine all [CLS] embdeddings
# final_embedding = torch.cat(embeddings, dim=0)

In [8]:
row_embeddings = []
for index, row in data.iterrows():
    text = row.iloc[:-2].apply(lambda x: f"{x}").to_string()
    inputs = bio_tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = bio_model(**inputs, output_hidden_states=True)
        last_hidden_state = outputs.hidden_states[-1]
        row_embedding = last_hidden_state[:, 0, :].squeeze()
        row_embeddings.append(row_embedding)
# convert list of tensors to numpy array
row_embeddings = torch.stack(row_embeddings).numpy()
np.shape(row_embeddings)

(170, 768)

In [9]:
import os
# Save the embeddings to a CSV file
output_dir = "embeddings"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "aml_embeddings.csv")
# combine the embeddings with the original labels
pd_embeddings = pd.DataFrame(row_embeddings)
embeddings_set = pd.concat([data.iloc[:, -2:], pd_embeddings], axis=1)
pd.DataFrame(embeddings_set).to_csv(output_path, index=False, header=True)

In [10]:
csv = pd.read_csv(output_path, sep=",", on_bad_lines="skip")
csv

,Survival,Death,0,1,2,3,4,5,6,7,...,758,759,760,761,762,763,764,765,766,767
0,245.0,1.0,-0.357964,0.014439,-0.607007,-0.479693,-0.388622,-0.005880,0.685194,-0.795156,...,0.302006,-0.100780,-0.301960,-0.407059,-0.225024,0.196594,-0.598340,-0.381602,0.342899,0.170169
1,212.0,1.0,-0.341357,0.199818,-0.492455,-0.407541,-0.319900,-0.011942,0.472954,-0.751148,...,0.352147,0.164420,-0.240853,-0.371359,0.000023,0.131608,-0.657566,-0.364837,0.486053,0.307420
2,31.0,1.0,-0.427426,0.014379,-0.448388,-0.242970,-0.503671,0.001477,0.562962,-0.714487,...,0.244776,0.052735,-0.250412,-0.503443,0.066732,0.030213,-0.637094,-0.324627,0.386945,0.362722
3,215.0,1.0,-0.232072,0.021510,-0.634586,-0.222306,-0.455781,0.037620,0.468638,-0.830539,...,0.248999,0.133043,-0.173256,-0.337102,-0.038984,0.032154,-0.490867,-0.172833,0.426780,0.466805
4,393.0,0.0,-0.268474,0.300018,-0.439341,-0.429752,-0.281321,-0.120318,0.532188,-0.782888,...,0.202076,-0.131384,-0.360854,-0.538226,0.024856,0.073008,-0.502723,-0.160276,0.294535,0.110275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,366.0,1.0,-0.434553,0.289929,-0.538732,-0.299896,-0.334135,-0.100635,0.381776,-0.608246,...,0.307471,0.048668,-0.298677,-0.549514,0.025923,0.172282,-0.556300,-0.229073,0.339185,0.257365
166,NaN,NaN,-0.335051,0.134798,-0.389931,-0.432373,-0.444002,0.071765,0.583645,-0.684876,...,0.306241,0.060266,-0.323684,-0.521405,0.090813,0.100819,-0.748978,-0.161036,0.435805,0.276709
167,212.0,0.0,-0.391729,0.300748,-0.546914,-0.317794,-0.304247,-0.060693,0.407570,-0.639861,...,0.305389,0.075536,-0.296897,-0.526657,0.061428,0.161922,-0.588598,-0.212573,0.398409,0.257948
168,334.0,0.0,-0.353560,0.103062,-0.503161,-0.414546,-0.351676,-0.002857,0.641335,-0.691787,...,0.267260,-0.024711,-0.338916,-0.434339,-0.030028,0.120505,-0.756004,-0.382593,0.464757,0.288696


# scBERT

In [11]:
# Install the required libraries
!pip install transformers pandas torch

# Import necessary libraries
import pandas as pd
import torch
import os
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification, AutoModel

Defaulting to user installation because normal site-packages is not writeable


In [12]:
# Use a pipeline as a high-level helper
pipe = pipeline("token-classification", model="havens2/scBERT_SER")

# Test the pipeline with a sample text
sample_text = "This is a test sentence to check scBERT."
result = pipe(sample_text)
print("Pipeline result:", result)

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Pipeline result: [{'entity': 'B-MethodName', 'score': 0.9369256, 'index': 8, 'word': 'sc', 'start': 33, 'end': 35}, {'entity': 'I-MethodName', 'score': 0.9552647, 'index': 9, 'word': '##bert', 'start': 35, 'end': 39}, {'entity': 'I-MethodName', 'score': 0.9356297, 'index': 10, 'word': '.', 'start': 39, 'end': 40}]


In [13]:
# Load model and tokenizer directly
tokenizer = AutoTokenizer.from_pretrained("havens2/scBERT_SER")
model = AutoModelForTokenClassification.from_pretrained("havens2/scBERT_SER")

# Test with a sample text
inputs = tokenizer(sample_text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
print("Raw logits from the model:", outputs.logits)

Raw logits from the model: tensor([[[ 6.8207e+00, -1.8527e+00,  3.5425e+00, -2.5288e+00, -7.8676e-01,
          -2.3096e+00, -1.0595e+00, -2.3098e+00,  6.7361e-01, -2.3066e+00,
          -2.8579e-01, -1.7613e+00,  1.4329e+00, -1.5983e+00,  1.0944e+00],
         [ 1.0271e+01, -5.8609e-01, -1.2012e-01, -1.3773e-01, -1.3827e+00,
          -1.0569e+00, -1.2172e+00, -1.3262e+00, -9.8027e-01, -1.3730e+00,
          -1.7418e+00, -1.0566e+00, -7.4307e-01, -9.0452e-01, -1.0170e+00],
         [ 1.0567e+01, -9.1079e-01, -5.3119e-01, -4.4759e-01, -1.3354e+00,
          -1.0729e+00, -1.1044e+00, -1.3796e+00, -1.0315e+00, -1.1238e+00,
          -1.6894e+00, -1.5108e+00, -9.7344e-01, -1.2494e+00, -5.8068e-01],
         [ 1.0646e+01, -7.8708e-01, -6.0165e-01, -5.7724e-01, -1.5686e+00,
          -8.3163e-01, -1.1573e+00, -1.1982e+00, -9.1989e-01, -1.3062e+00,
          -1.7632e+00, -1.5085e+00, -1.1754e+00, -1.0294e+00, -7.2188e-01],
         [ 9.7495e+00, -5.5458e-01, -1.1337e+00, -2.4265e-02, -1.4182

In [14]:
# convert dataframe to string
print(data.iloc[0, :-2]) # remove the last two columns (label)
embedding_text_2 = data.iloc[0, :-2].apply(lambda x: f"{x}").to_string()
print()
print(embedding_text_2)

Unnamed: 0
X..100130426   -16.60964
X..100133144    4.191342
X..100134869   -16.60964
X..26823       -16.60964
X..340602      -16.60964
                  ...   
hsa.mir.95     -16.60964
hsa.mir.96     -16.60964
hsa.mir.98      6.223134
hsa.mir.99a     6.332759
hsa.mir.99b      9.33715
Name: 0, Length: 11705, dtype: object

Unnamed: 0
X..100130426              -16.609640474436812
X..100133144                4.191342344959819
X..100134869              -16.609640474436812
X..26823                  -16.609640474436812
X..340602                 -16.609640474436812
X..388795                    3.40284630657414
X..391343                 -16.609640474436812
X..391714                 -16.609640474436812
X..441362                 -16.609640474436812
X..652919                   5.228819075214507
X..728603                 -16.609640474436812
X..728788                  1.9434375348274688
X..729884                 -16.609640474436812
X..90288                  -16.609640474436812
A2BP1.54715         

In [15]:
# Generate embeddings using scBERT
row_embeddings_2 = []
for index, row in data.iterrows():
    # Combine row data into a single text string (exclude the last two columns if they are labels/metadata)
    text = row.iloc[:-2].apply(str).to_string()
    
    # Tokenize the text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    
    # Perform inference with hidden state outputs
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        last_hidden_state = outputs.hidden_states[-1]  # Extract the last Transformer layer's hidden states
    
    # Extract the [CLS] token embedding
    cls_embedding = last_hidden_state[:, 0, :].squeeze()
    
    # Append the embedding to the list
    row_embeddings_2.append(cls_embedding)

# Convert list of tensors to a NumPy array
row_embeddings_2 = torch.stack(row_embeddings_2).numpy()

In [16]:
# Save embeddings to a CSV file
output_dir = "embeddings"
os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists
output_path = os.path.join(output_dir, "scbert_embeddings.csv")

# Create a DataFrame for the embeddings
pd_embeddings = pd.DataFrame(row_embeddings_2)

# Combine with original metadata or labels (last two columns)
embeddings_set = pd.concat([data.iloc[:, -2:], pd_embeddings], axis=1)

# Save to CSV
embeddings_set.to_csv(output_path, index=False, header=True)
print(f"Embeddings saved to: {output_path}")

Embeddings saved to: embeddings\scbert_embeddings.csv


In [17]:
# Reload saved CSV for validation
csv = pd.read_csv(output_path)
print("Preview of saved embeddings:")
print(csv.head())

Preview of saved embeddings:
   Survival  Death         0         1         2         3         4  \
0     245.0    1.0 -0.586473  1.126267 -1.233645  0.337535 -0.079401   
1     212.0    1.0 -0.546070  1.114134 -1.255844  0.250813 -0.127561   
2      31.0    1.0 -0.646991  0.014289 -0.577768  0.601122 -0.025645   
3     215.0    1.0 -0.547844 -0.053747 -0.652106  0.633017  0.135609   
4     393.0    0.0 -0.662480  0.025126 -0.720215  0.538170  0.128502   

          5         6         7  ...       758       759       760       761  \
0 -0.575628  0.756565 -0.018674  ...  0.071955 -0.403486 -0.862712 -0.348951   
1 -0.700103  0.818661 -0.132997  ...  0.251329 -0.415166 -0.942010 -0.304763   
2 -0.311130  0.594801 -0.226399  ... -0.467999 -0.986424 -1.025536  0.187038   
3 -0.406480  0.513530 -0.305261  ... -0.515952 -1.051046 -0.877564  0.180690   
4 -0.317073  0.541753 -0.218624  ... -0.517045 -1.057750 -1.061812  0.167840   

        762       763       764       765       766      

# END